# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [63]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [64]:
geoapify_key

'6b77b9c4d9b74c97a2e8c887d16ba817'

In [65]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,faya,18.3851,42.4509,22.06,35,21,2.60,SA,1666108228
1,1,farsund,58.0948,6.8047,13.30,100,0,7.65,NO,1666108228
2,2,new norfolk,-42.7826,147.0587,11.72,58,12,1.34,AU,1666108230
3,3,jamestown,42.0970,-79.2353,5.77,77,100,9.77,US,1666107934
4,4,lanzhou,36.0564,103.7922,14.53,48,59,1.20,CN,1666108230


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [66]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points('Lng','Lat',
                                      geo=True,
                                      tiles='OSM',
                                      size = 'Humidity',
                                      color = 'Country',
                                      alpha = 0.5,
                                      hover_cols = 'City',
                                      frame_width = 800,
                                      frame_height = 600)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Country,Humidity,City)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [67]:
# Narrow down cities that fit criteria and drop any results with null values
narrow_city_df = city_data_df[(city_data_df['Max Temp'] >= 21) & 
                              (city_data_df['Max Temp'] <= 27) & 
                              (city_data_df['Wind Speed'] < 3) &
                              (city_data_df['Cloudiness'] < 10)
                              ]

# Drop any rows with null values
narrow_city_df.dropna()

# Display sample data
narrow_city_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
51,51,hilo,19.7297,-155.0900,26.27,83,0,2.57,US,1666108260
63,63,banda,25.4833,80.3333,24.62,52,0,2.68,IN,1666108268
81,81,makakilo city,21.3469,-158.0858,21.66,81,0,2.57,US,1666108282
211,211,laguna,38.4210,-121.4238,21.67,79,0,2.06,US,1666108364
240,240,tikaitnagar,26.9500,81.5833,23.56,59,0,0.35,IN,1666108378
265,265,san quintin,30.4833,-115.9500,21.20,74,0,1.37,MX,1666108394
317,317,salalah,17.0151,54.0924,25.49,59,7,1.67,OM,1666108424
334,334,kailua,21.4022,-157.7394,22.47,89,1,1.18,US,1666108432
340,340,santa rosalia,27.3167,-112.2833,24.62,56,0,0.74,MX,1666108436
363,363,narwar,25.6500,77.9000,22.35,55,0,1.29,IN,1666108449


### Step 3: Create a new DataFrame called `hotel_df`.

In [68]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = narrow_city_df[['City','Country','Lat','Lng','Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
51,hilo,US,19.7297,-155.0900,83,
63,banda,IN,25.4833,80.3333,52,
81,makakilo city,US,21.3469,-158.0858,81,
211,laguna,US,38.4210,-121.4238,79,
240,tikaitnagar,IN,26.9500,81.5833,59,
265,san quintin,MX,30.4833,-115.9500,74,
317,salalah,OM,17.0151,54.0924,59,
334,kailua,US,21.4022,-157.7394,89,
340,santa rosalia,MX,27.3167,-112.2833,56,
363,narwar,IN,25.6500,77.9000,55,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [76]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel"
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = hotel_df.loc[index, "Lat"]
    lon = hotel_df.loc[index, "Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    params["apiKey"] = geoapify_key

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url,params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
hilo - nearest hotel: Dolphin Bay Hotel
banda - nearest hotel: #acnindiafy21
makakilo city - nearest hotel: Embassy Suites by Hilton Oahu Kapolei
laguna - nearest hotel: Holiday Inn Express & Suites
tikaitnagar - nearest hotel: No hotel found
san quintin - nearest hotel: Jardines Hotel
salalah - nearest hotel: Muscat International Hotel
kailua - nearest hotel: No hotel found
santa rosalia - nearest hotel: Sol y Mar
narwar - nearest hotel: No hotel found
roebourne - nearest hotel: No hotel found
stolac - nearest hotel: Bregava
samandag - nearest hotel: Almina Apart Hotel
lahij - nearest hotel: No hotel found
guerrero negro - nearest hotel: Plaza sal paraiso


,City,Country,Lat,Lng,Humidity,Hotel Name
51,hilo,US,19.7297,-155.0900,83,Dolphin Bay Hotel
63,banda,IN,25.4833,80.3333,52,#acnindiafy21
81,makakilo city,US,21.3469,-158.0858,81,Embassy Suites by Hilton Oahu Kapolei
211,laguna,US,38.4210,-121.4238,79,Holiday Inn Express & Suites
240,tikaitnagar,IN,26.9500,81.5833,59,No hotel found
265,san quintin,MX,30.4833,-115.9500,74,Jardines Hotel
317,salalah,OM,17.0151,54.0924,59,Muscat International Hotel
334,kailua,US,21.4022,-157.7394,89,No hotel found
340,santa rosalia,MX,27.3167,-112.2833,56,Sol y Mar
363,narwar,IN,25.6500,77.9000,55,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [79]:
%%capture --no-display

# Configure the map plot
map_plot_final = hotel_df.hvplot.points('Lng','Lat',
                                      geo=True,
                                      tiles='OSM',
                                      size = 'Humidity',
                                      color = 'City',
                                      alpha = 0.5,
                                      hover_cols = ['Hotel Name','Country'],
                                      frame_width = 800,
                                      frame_height = 600)

# Display the map
map_plot_final

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)